In [1]:
from contextlib import contextmanager

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base, declared_attr
from sqlalchemy.orm import sessionmaker


In [2]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
Base = declarative_base()

In [4]:
class ModelBase:
    
    @declared_attr
    def __tablename__(cls):
        return cls.__name__.lower()

    id =  Column(Integer, primary_key=True)
    
    def __init__(self):
        self.Session = sessionmaker(bind=engine)
        self.session = None
        
    @contextmanager
    def session_scope(self):
        """ Provide a transactional scope around a series of operations.
        """
        self.session = self.Session()
        try:
            yield self.session
            self.session.commit()
        except:
            self.session.rollback()
            raise
        finally:
            self.session.close()
            
    def save(self):
        with self.session_scope() as session:
            session.add(self)

In [5]:
class User(ModelBase, Base):
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __init__(self, name, fullname, password):
        super().__init__()            
        self.name = name
        self.fullname = fullname
        self.password = password

In [6]:
Base.metadata.create_all(engine)

2016-09-25 16:11:11,529 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-09-25 16:11:11,530 INFO sqlalchemy.engine.base.Engine ()
2016-09-25 16:11:11,531 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-09-25 16:11:11,531 INFO sqlalchemy.engine.base.Engine ()
2016-09-25 16:11:11,532 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user")
2016-09-25 16:11:11,533 INFO sqlalchemy.engine.base.Engine ()
2016-09-25 16:11:11,534 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2016-09-25 16:11:11,534 INFO sqlalchemy.engine.base.Engine ()
2016-09-25 16:11:11,535 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
user = User(name='Jason', fullname='Jason Myers', password='cookies')

sessionmaker(class_='Session',autoflush=True, bind=Engine(sqlite:///:memory:), expire_on_commit=True, autocommit=False)
None


In [8]:
user.save()

2016-09-25 16:11:11,543 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-25 16:11:11,544 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, password) VALUES (?, ?, ?)
2016-09-25 16:11:11,544 INFO sqlalchemy.engine.base.Engine ('Jason', 'Jason Myers', 'cookies')
2016-09-25 16:11:11,545 INFO sqlalchemy.engine.base.Engine COMMIT
